<a href="https://colab.research.google.com/github/snakeway8/Projects/blob/main/api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests gspread oauth2client flask

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta

# Set up access to Google Sheets
def authenticate_google_sheets():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('credential.json', scope)
    client = gspread.authorize(creds)
    return client

# Function to get the exchange rate
def get_exchange_rate(date):
    formatted_date = date.strftime('%d.%m.%Y')  # Format for PrivatBank API
    url = f"https://api.privatbank.ua/p24api/exchange_rates?json&date={formatted_date}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for rate in data['exchangeRate']:
            if rate.get('currency') == 'USD':
                return rate['saleRate']
    return None

# Function to update data in Google Sheets
def update_google_sheet(sheet, date, rate):
    existing_dates = sheet.col_values(1)  # Assuming the date is in the first column
    if date.strftime('%Y-%m-%d') not in existing_dates:
        sheet.append_row([date.strftime('%Y-%m-%d'), rate])
        print(f"Data for {date.strftime('%Y-%m-%d')} updated: {rate}")
    else:
        print(f"Data for {date.strftime('%Y-%m-%d')} already exists, skipping.")

# Main function to update rates for a date range
def update_rates(update_from="2023-01-01", update_to="2023-01-07"):
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(update_from, date_format)
    end_date = datetime.strptime(update_to, date_format)

    client = authenticate_google_sheets()
    sheet = client.open("currency").sheet1

    current_date = start_date
    while current_date <= end_date:
        rate = get_exchange_rate(current_date)
        if rate:
            update_google_sheet(sheet, current_date, rate)
        current_date += timedelta(days=1)

# Example run with parameters
if __name__ == "__main__":
    update_rates("2024-01-01", "2024-01-07")


Data for 2018-03-13 updated: 26.2
Data for 2018-03-14 updated: 26.5
Data for 2018-03-15 updated: 26.65
Data for 2018-03-16 updated: 26.65
Data for 2018-03-17 updated: 26.65
Data for 2018-03-18 updated: 26.65
Data for 2018-03-19 updated: 26.65
Data for 2018-03-20 updated: 26.5
Data for 2018-03-21 updated: 26.5
Data for 2018-03-22 updated: 26.4
Data for 2018-03-23 updated: 26.4
Data for 2018-03-24 updated: 26.4
Data for 2018-03-25 updated: 26.4
Data for 2018-03-26 updated: 26.4
Data for 2018-03-27 updated: 26.4
Data for 2018-03-28 updated: 26.5
Data for 2018-03-29 updated: 26.55
Data for 2018-03-30 updated: 26.35
Data for 2018-03-31 updated: 26.35
Data for 2018-04-01 updated: 26.35
Data for 2018-04-02 updated: 26.25
Data for 2018-04-03 updated: 26.35
Data for 2018-04-04 updated: 26.35
Data for 2018-04-05 updated: 26.2
Data for 2018-04-06 updated: 26.2
Data for 2018-04-07 updated: 26.2
Data for 2018-04-08 updated: 26.2
Data for 2018-04-09 updated: 26.2
Data for 2018-04-10 updated: 26.15
D